# Парсер мероприятий с it-events.com

## Импорты, листай ниже

In [23]:
import bs4, requests
import re
import pandas as pd
import meetup.api
import numpy as np
import seaborn

from matplotlib import pyplot as plt
from datetime import datetime

## Вспомогательные функции

In [56]:
def get_content(x):
    x = str(x.contents[0]).replace('\n', '').replace(u'\xa0', '')
    x = re.sub(r'(<.*?>)', '', x)
    """
    .replace('<strong>', '') \
    .replace('</strong>', '') \
    .replace('<em>', '') \
    .replace('<\em>', '')
    if '<img' in x:
        return ''
    if '<a' in x:
        return ''
    """
    return x

def get_text(soup):
    all_p = list(map(get_content, soup.findAll('p')))
    all_li = list(map(get_content, soup.findAll('li')))
    final_text = ' '.join(all_p + all_li)
    return final_text

month_map = {
    "января": 1,
    "февраля": 2,
    "марта": 3, 
    "апреля": 4,
    "мая": 5,
    "июня": 6, 
    "июля": 7,
    "августа": 8,
    "сентября": 9,
    "октября": 10,
    "ноября": 11,
    "декабря": 12
}

def transform_date(date): # list -> str
    date[1] = month_map[date[1]]
    return date

def get_datetime_from_date(date): # str -> list
    date.replace(' - ', '-')
    if '-' not in date:
        return transform_date(date.split(' ')), []
    start, end = date.split('-')
    start = start.split(' ')
    end = end.split(' ')
    while '' in start:
        start.remove('')
    while '' in end:
        end.remove('')
    if len(start) == 1:
        start.append(end[1])
        start.append(end[2])
    start = transform_date(start)
    end = transform_date(end)
    return start, end

## Функция, которая парсит уже прошедшие мероприятия
start_page и end_page - страницы на it-events, с какой по какую нужно парсить

In [57]:
# get past events
def parse_it_events(end_page, start_page=1, path='https://it-events.com/?type=past&page='):
    cities = []
    names = []
    ids = []
    descrs = []
    dates_start = []
    dates_end = []
    source = []
    costs = []
    types = []
    links = []
    for j in range(start_page, end_page):
        print(j)
        path = path + str(j)
        root = bs4.BeautifulSoup(requests.get(path).text)
        for ev in root.findAll('div', 'event-list-item'):
            event_name = ev.findAll('a', 'event-list-item__title')[0].contents[0].replace('\n', '')
            event_id = ev.findAll('a', 'event-list-item__title')[0].get('href').split('/')[-1]
            try:
                event_city = ev.findAll('div', 'event-list-item__info event-list-item__info_location')[0].contents[0]
                event_city = event_city.replace('\n', '')
            except:
                event_city = ''
                continue
            try:
                event_date = ev.findAll('div', 'event-list-item__info')
                event_date = event_date[0].contents[0].replace('\n', '')
                
            except:
                event_date = ''
                continue
            
            try:
                tmp = ev.findAll('div', 'event-list-item__type')[0].contents[0]
                event_type, event_cost = tmp.replace('\n', '').split(' / ')
            except:
                event_cost = ''
                event_type = ''
            
            event_path = 'https://it-events.com/events/' + str(event_id)
            page = bs4.BeautifulSoup(requests.get(event_path).text)
            
            try:
                source_link = page.findAll('a', 'event-common-item__link event-common-item__link_ellipsis')[0].get('href')
            except:
                source_link = ''
            
            descr = get_text(page)
            names.append(event_name)
            ids.append(event_id)
            descrs.append(descr)
            cities.append(event_city)
            
            try:
                date_start, date_end = get_datetime_from_date(event_date)
            except:
                date_start, date_end = '', ''
            
            dates_start.append(date_start)
            dates_end.append(date_end)
            source.append('it-events')
            costs.append(event_cost)
            types.append(event_type)
            links.append(source_link)
    data = {'title': names,
            'source': source,
            'id': ids,
            'description': descrs,
            'date_start': dates_start, 
            'date_end': dates_end,
            'city': cities,
            'cost': costs,
            'type': types,
            'link': links}
    return pd.DataFrame(data=data)

## Парсинг новых мероприятий
Основной код тот же, но разбит на функции и мероприятия дополнительно фильтруются по дате последнего запуска

In [58]:
# Parse single page
def parse_page(root):
    events = []
    for ev in root.findAll('div', 'event-list-item'):
        event_name = ev.findAll('a', 'event-list-item__title')[0].contents[0].replace('\n', '')
        event_id = ev.findAll('a', 'event-list-item__title')[0].get('href').split('/')[-1]
        try:
            event_city = ev.findAll('div', 'event-list-item__info event-list-item__info_location')[0].contents[0]
            event_city = event_city.replace('\n', '')
        except:
            event_city = ''
        try:
            event_date = ev.findAll('div', 'event-list-item__info')
            event_date = event_date[0].contents[0].replace('\n', '')
                
        except:
            event_date = ''
        try:
            tmp = ev.findAll('div', 'event-list-item__type')[0].contents[0]
            event_type, event_cost = tmp.replace('\n', '').split(' / ')
        except:
            event_cost = ''
            event_type = ''
            
        event_path = 'https://it-events.com/events/' + str(event_id)
        page = bs4.BeautifulSoup(requests.get(event_path).text)
            
        try:
            source_link = page.findAll('a', 'event-common-item__link event-common-item__link_ellipsis')[0].get('href')
        except:
            source_link = ''
            
        descr = get_text(page)
        try:
            date_start, date_end = get_datetime_from_date(event_date)
        except:
            date_start, date_end = '', ''    
        
        event = {
            'name': event_name,
            'source': 'it-events',
            'id': event_id,
            'description': descr,
            'city': event_city,
            'date_start': date_start,
            'date_end': date_end,
            'cost': event_cost,
            'type': event_type,
            'link': source_link
        }
        events.append(event)
    return events
    
# Go through all pages
def parse_new_events(path='https://it-events.com/?type=upcoming&page='):
    i = 1
    events = []
    root = bs4.BeautifulSoup(requests.get(path + str(i)).text)
    while len(root.findAll('div', 'event-list-item')) > 0:
        events += parse_page(root)
        i += 1
        root = bs4.BeautifulSoup(requests.get(path + str(i)).text)
    return events

# Date filter
def filter_events(events, last_date):
    for ev in events.copy():
        try:
            d, m, y = list(map(int, ev['date_start']))
        except:
            events.remove(ev)
            continue
        if datetime(y, m, d) < last_date:
            events.remove(ev)
    return events

# Convert dict to pandas
def to_pandas_df(events):
    names = []
    sources = []
    ids = []
    descriptions = []
    cities = []
    dates_start = []
    dates_end = []
    costs = []
    types = []
    links = []
    for ev in events:
        names.append(ev['name'])
        sources.append(ev['source'])
        ids.append(ev['id'])
        descriptions.append(ev['description'])
        cities.append(ev['city'])
        dates_start.append(ev['date_start'])
        dates_end.append(ev['date_end'])
        costs.append(ev['cost'])
        types.append(ev['type'])
        links.append(ev['link'])
    data = {
        'title': names,
        'source': sources,
        'id': ids,
        'description': descriptions,
        'date_start': dates_start, 
        'date_end': dates_end,
        'city': cities,
        'cost': costs,
        'type': types,
        'link': links
    }
    df = pd.DataFrame(data=data)
    return df
        
# Main function
def get_new_events(date):
    events_list = parse_new_events()
    filtered_events_list = filter_events(events_list, date)
    df = to_pandas_df(filtered_events_list)
    return df

## Тестовый запуск

In [59]:
%%time
events = get_new_events(datetime(2019, 10, 22))
events.head()

CPU times: user 5.6 s, sys: 156 ms, total: 5.76 s
Wall time: 1min 15s


In [60]:
# Also you can save it to csv
events.to_csv('new_events.csv')

## Парсинг страниц прошедших мероприятий

In [61]:
%%time
# 2 pages
data = parse_it_events(2)

1
CPU times: user 567 ms, sys: 24.5 ms, total: 591 ms
Wall time: 8.34 s


In [5]:
data.head()

,title,source,id,description,date_start,date_end,city,cost,type,link
0,PeerLab Киров Meetup,it-events,15911,PeerLab – В этот раз кировский PeerLab совме...,"[13, 7, 2019]","[14, 7, 2019]","Слободской, Россия",Платное,Митап,https://hutorium.timepad.ru/event/962270/
1,День открытых дверей магистратуры «Цифровая эк...,it-events,16102,"Всех, кто планирует в 2019 году продолжить сво...","[13, 7, 2019]",[],"Москва, Россия",Бесплатное,Курс,
2,День открытых дверей в MyAcademy,it-events,15938,Марафон бесплатных мастер-классов по интернет-...,"[13, 7, 2019]",[],"Москва, Россия",Бесплатное,Мастер-класс,https://myacademy.ru/kursi/den-otkritih-dverei...
3,GodelTech Meetup BREST #1,it-events,15941,Почему стоит сходить? А как же узнать о пут...,"[13, 7, 2019]",[],"Брест, Беларусь",Бесплатное,Митап,https://www.godeltech.com/events/godel-tech-me...
4,Конференция 8P,it-events,15715,13 июля в Одессе 1800+ участников На протяж...,"[13, 7, 2019]",[],"Одесса, Украина",Платное,Конференция,http://bit.ly/ITEVENT_8P


In [62]:
data.to_csv('past_events.csv')

# Недопарсинг Timepad

In [105]:
def get_path(limit=20):
    return 'https://api.timepad.ru/v1/events.json?' + \
    'limit='+str(limit) + '&fields=location&sort=+starts_at'

path = get_path(100)
print(path)
import json
t = requests.get(path).text
# print(t)
collection = json.loads(t)
try:
    print(len(collection['values']))
except:
    print(collection)

https://api.timepad.ru/v1/events.json?limit=100&fields=location&sort=+starts_at
100
